In [65]:
!pip install spacy tabula https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.5.0/en_core_web_sm-3.5.0-py3-none-any.whl


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 13.0 MB/s eta 0:00:00 0:00:01


numpy version: 1.26.0
pandas version: 2.2.3
scipy version: 1.11.3


In [7]:
# Packages
import os
import re
import pdfplumber
import numpy
import pandas as pd
import spacy
from datetime import datetime

In [8]:
# Load spaCy English model
nlp = spacy.load("en_core_web_sm")

# Load the provider info CSV for ending keywords
provider_info = pd.read_csv('provider.csv')

# Load the company info CSV for ticker validation and company metadata
company_info = pd.read_csv('company_info.csv')  # Replace with the actual path

company_info = company_info.drop_duplicates(subset='Ticker Symbol')

# Create a dictionary to map ticker symbols to company name and industry
ticker_map = company_info.set_index('Ticker Symbol')[['Company Name', 'Industry']].to_dict(orient='index')

# Testing

In [64]:
import pdfplumber

def extract_words_with_formatting(page):
    """
    Extracts words along with their formatting details such as font size and font name.

    Args:
        page (pdfplumber.Page): A single page from the PDF.

    Returns:
        List[Dict]: A list of dictionaries containing words and their formatting details.
    """
    # Extract words with their bounding boxes
    words = page.extract_words(extra_attrs=["fontname", "size"])

    formatted_words = []
    for word in words:
        formatted_words.append({
            "word": word["text"],
            "font": word.get("fontname", "Unknown"),
            "size": word.get("size", "Unknown"),
            "x0": word["x0"],
            "x1": word["x1"],
            "top": word["top"],
            "bottom": word["bottom"]
        })
    return formatted_words


# Example usage with pdfplumber
pdf_path = "/Users/oskarroeske/Masterthesis/preprocessing/reports/20170718_Needham_AMZN_Prime_Day_2017_Bigger_and_Better.pdf"

with pdfplumber.open(pdf_path) as pdf:
    for page_number, page in enumerate(pdf.pages, start=1):
        print(f"Page {page_number}:")
        formatted_words = extract_words_with_formatting(page)
        for word_info in formatted_words:
            print(
                f"Word: '{word_info['word']}', Font: {word_info['font']}, Size: {word_info['size']}, "
                f"Position: ({word_info['x0']}, {word_info['top']} - {word_info['x1']}, {word_info['bottom']})"
            )


Page 1:
Word: 'July', Font: EACHCI+Cambria-Italic, Size: 9.0, Position: (522.03700342, 57.559999840000046 - 536.06800342, 66.55999984000005)
Word: '11,', Font: EACHCI+Cambria-Italic, Size: 9.0, Position: (538.04800342, 57.559999840000046 - 549.3430034200001, 66.55999984000005)
Word: '2017', Font: EACHCI+Cambria-Italic, Size: 9.0, Position: (551.3230034200001, 57.559999840000046 - 570.33100342, 66.55999984000005)
Word: 'Market', Font: EACHCH+Cambria-Bold, Size: 10.0, Position: (421.512, 622.7410002400001 - 454.582, 632.7410002400001)
Word: 'Data', Font: EACHCH+Cambria-Bold, Size: 10.0, Position: (456.782, 622.7410002400001 - 478.172, 632.7410002400001)
Word: 'Price', Font: EACHCG+Cambria, Size: 8.0, Position: (421.512, 655.71000027 - 438.98400000000004, 663.71000027)
Word: '(07/10/2017)', Font: EACHCG+Cambria, Size: 8.0, Position: (440.744, 655.71000027 - 490.072, 663.71000027)
Word: '$996.47', Font: EACHCG+Cambria, Size: 8.0, Position: (547.34899899, 655.71000027 - 575.14899899, 663.71

In [93]:
pattern = r"E[A-Z]+[+]Cambria(-Bold)?"

print(re.match(pattern,"EDNIXB+Cambria-Bold"))


<re.Match object; span=(0, 19), match='EDNIXB+Cambria-Bold'>


In [130]:
import re

# Define a mapping for providers and their patterns
patterns = {
    "BGC Partners": {
        "primary_price_pattern": r"Price Target \(\$\) (\d+(\.\d+)?)",
        "primary_rating_pattern": r"([A-Za-z]+) \(\w+,",
        "primary_ending_pattern" :r"Disclosures Appendix",
        "primary_font_size_pattern":7.92,
        "secondary_font_size_pattern":10.0,
        "primary_font_type_pattern": r"Tahoma(-Bold)?",
        "secondary_font_type_pattern": "Not Available"
    },
    "Needham": {
        "primary_price_pattern": r"Price Target: \$(\d+(\,\d+)?)",
        "secondary_price_pattern": r"PRICE TARGET: \$(\d+(\,\d+)?)",
        "primary_rating_pattern": r"\b(BUY|HOLD|SELL|OVERWEIGHT|UNDERPERFORM)\b",
        "secondary_rating_pattern": r"Rating (\w+)",
        "primary_ending_pattern" :r"Analyst Certification",
        "primary_font_size_pattern":10.0,
        "secondary_font_size_pattern":10.0,
        "primary_font_type_pattern": r"E[A-Z]+[+]Cambria(-Bold)?",
        "secondary_font_type_pattern": "Not Available",
    },
    "BTIG": {
        "primary_price_pattern": r"\$(\d+(\.\d+)?) 12 month target ",
        "primary_rating_pattern": r"\b(BUY|HOLD|SELL|OVERWEIGHT|UNDERPERFORM)\b",
        "primary_ending_pattern" :r"Appendix: Analyst Certification and Other Important Disclosures",
        "primary_font_size_pattern":9.96,
        "secondary_font_size_pattern":10.0,
        "primary_font_type_pattern": r"[A-Z]+[+]Corbel(,Bold)?(,-Italic)?",
        "secondary_font_type_pattern": "Not Available"

    },
    "Wells Fargo": {
        "primary_price_pattern": r"/\$(\d+(\.\d+)?)",
        "secondary_price_pattern": r"Price Target\/Prior: \$(\d+(\.\d+)?)",  
        "primary_rating_pattern": r"([A-Za-z]+)/\$",
        "secondary_rating_pattern": r"Rating (\w+)",
        "primary_ending_pattern" :r"Required Disclosures",
        "primary_font_size_pattern":7.92,
        "secondary_font_size_pattern":10.0,
        "primary_font_type_pattern": r"Tahoma(-Bold)?",
        "secondary_font_type_pattern": "Not Available"
    },
    "Barclays": {
        "primary_price_pattern": r"Price Target USD (\d+(\.\d+)?)",
        "primary_rating_pattern": r"Stock Rating ([A-Za-z]+)",
        "primary_ending_pattern" :r"ANALYST(S) CERTIFICATION(S)",
        "primary_font_size_pattern":7.92,
        "secondary_font_size_pattern":10.0,
        "primary_font_type_pattern": r"Tahoma(-Bold)?",
        "secondary_font_type_pattern": "Not Available"
    },
    "JP Morgan": {
        "primary_price_pattern": r"Price Target \([A-Za-z0-9]+\): \$(\d+(\.\d+)?)",
        "primary_rating_pattern": r"([A-Za-z]+) [A-Za-z]+",
        "primary_ending_pattern" :r"Analyst Certification",
        "primary_font_size_pattern":7.92,
        "secondary_font_size_pattern":10.0,
        "primary_font_type_pattern": r"Tahoma(-Bold)?",
        "secondary_font_type_pattern": "Not Available"
    },
    "Brean Capital LLC": {
        "primary_price_pattern": r"PT: \$ (\d+(\.\d+)?)",
        "primary_rating_pattern": r"([A-Za-z]+) PT:\$",
        "primary_ending_pattern" :r"Analyst Certification",
        "primary_font_size_pattern":7.92,
        "secondary_font_size_pattern":10.0,
        "primary_font_type_pattern": r"Tahoma(-Bold)?",
        "secondary_font_type_pattern": "Not Available"
    },
    "Hilliard Lyons": {
        "primary_price_pattern": r"Price Target (\d+(\.\d+)?)",
        "primary_rating_pattern": r"-- ([A-Za-z]+) --",
        "primary_ending_pattern" :r"Analyst Certification"
    },
    "Alliance Global Partners": {
        "primary_price_pattern": r"Price Target \$(\d+(\.\d+)?)",
        "primary_rating_pattern": r"([A-Za-z]+) \(Ticker:",
        "primary_ending_pattern" :r"Imporant Research Disclosures",
        "primary_font_size_pattern":7.92,
        "secondary_font_size_pattern":10.0,
        "primary_font_type_pattern": r"Tahoma(-Bold)?",
        "secondary_font_type_pattern": "Not Available"
    },
    "Mizuho Securities": {
        "primary_price_pattern": r"Price Target \$(\d+(\.\d+)?)",
        "primary_rating_pattern": r"Rating ([A-Za-z]+)",
        "primary_ending_pattern" :r"IMPORTANT DISCLOSURES",
        "primary_font_size_pattern":7.92,
        "secondary_font_size_pattern":10.0,
        "primary_font_type_pattern": r"Tahoma(-Bold)?",
        "secondary_font_type_pattern": "Not Available"
    },
    "Gilford Securities Inc": {
        "primary_price_pattern": r"\, \$(\d+(\.\d+)?)", 
        "primary_rating_pattern": r"Rated: ([A-Za-z]+)",
        "secondary_rating_pattern": r"\b(BUY|HOLD|SELL|OVERWEIGHT|UNDERPERFORM)\b",
        "primary_ending_pattern" :r"ANALYST CERTIFICATION",
        "primary_font_size_pattern":7.92,
        "secondary_font_size_pattern":10.0,
        "primary_font_type_pattern": r"Tahoma(-Bold)?",
        "secondary_font_type_pattern": "Not Available"

    },
    "Deutsche Bank": {
        "primary_price_pattern": r"Price Target \(USD\) (\d+(\.\d+)?)",
        "primary_rating_pattern": r"Rating ([A-Za-z]+)",
        "primary_ending_pattern" :r"Appendix 1",
        "primary_font_size_pattern":7.92,
        "secondary_font_size_pattern":10.0,
        "primary_font_type_pattern": r"Tahoma(-Bold)?",
        "secondary_font_type_pattern": "Not Available"
    },
    "Pivotal Research Group": {
        "primary_price_pattern": r"Target Price: \$(\d+(,\d+)?)",
        "primary_rating_pattern": r"RATING: ([A-Za-z]+)",
        "primary_ending_pattern" :r"Appendix: Important Disclosures",
        "primary_font_size_pattern":7.92,
        "secondary_font_size_pattern":10.0,
        "primary_font_type_pattern": r"Tahoma(-Bold)?",
        "secondary_font_type_pattern": "Not Available"
    },
    "Spartan Capital Securities LLC": {
        "primary_price_pattern": r"T \$(\d+(\.\d+)?)",
        "primary_rating_pattern": r"([A-Za-z]+)",
        "primary_ending_pattern" :r"Important Disclosures",
        "primary_font_size_pattern":7.92,
        "secondary_font_size_pattern":10.0,
        "primary_font_type_pattern": r"Tahoma(-Bold)?",
        "secondary_font_type_pattern": "Not Available"
    },
    "Cascend Securities -Historical-": {
        "primary_price_pattern": r"Price target: \$(\d+(\.\d+)?)",
        "primary_rating_pattern": r"Rating: ([A-Za-z]+)",
        "primary_ending_pattern" :r"Disclosures: ",
        "primary_font_size_pattern":7.92,
        "secondary_font_size_pattern":10.0,
        "primary_font_type_pattern": r"Tahoma(-Bold)?",
        "secondary_font_type_pattern": "Not Available"
    },
    "Phillip Securities": {
        "primary_price_pattern": r"TARGET PRICE USD (\d+(\.\d+)?)",
        "primary_rating_pattern": r"\b(BUY|HOLD|SELL|OVERWEIGHT|UNDERPERFORM)\b \(",
        "primary_ending_pattern" :r"Contact Information",
        "primary_font_size_pattern":7.92,
        "secondary_font_size_pattern":10.0,
        "primary_font_type_pattern": r"Tahoma(-Bold)?",
        "secondary_font_type_pattern": "Not Available"
    },
    "FinTrust Investment Advisors": {
        "primary_price_pattern": r"Target Price: \$(\d+(,\d+)?)",
        "primary_rating_pattern": r"Fintrust Rating: ([A-Za-z]+)",
        "primary_ending_pattern" :r"Important Disclosures:",
        "primary_font_size_pattern":7.92,
        "secondary_font_size_pattern":10.0,
        "primary_font_type_pattern": r"Tahoma(-Bold)?",
        "secondary_font_type_pattern": "Not Available"
    },
    "IBI Investment House":{
        "primary_price_pattern": r"Price target: \$(\d+(,\d+)?)",
        "primary_rating_pattern": r"Recommendation: ([A-Za-z]+)",
        "primary_ending_pattern" :r"Disclosures",
        "primary_font_size_pattern":7.92,
        "secondary_font_size_pattern":10.0,
        "primary_font_type_pattern": r"Tahoma(-Bold)?",
        "secondary_font_type_pattern": "Not Available"
    }
}

In [131]:
def check_validity(paragraph):
    # Parse the paragraph
    paragraph = re.sub(r"\s+", " ", paragraph).strip()
    doc = nlp(paragraph)
        
    for sent in doc.sents:
        has_verb = False
        has_subject = False
        
        for token in sent:
            print(f"Token: {token.text}, POS: {token.pos_}, Dep: {token.dep_}")
            # Check for a verb
            if token.pos_ in {"VERB", "AUX"}:
                has_verb = True
            # Check for a subject
            if token.dep_ in {"nsubj", "nsubjpass"}:
                has_subject = True
        
        # If both a verb and a subject are found, the sentence is valid
        if has_verb and has_subject:
            print("Verb and Subject")
            return True
        
        # At least one word with 5+ letters and all upper case
        if re.search(r"[A-Z]{5,}", paragraph):
            return True
    return False

print(check_validity("Source: Company reports; BTIG"))


Token: Source, POS: NOUN, Dep: dep
Token: :, POS: PUNCT, Dep: punct
Token: Company, POS: NOUN, Dep: nsubj
Token: reports, POS: VERB, Dep: ROOT
Token: ;, POS: PUNCT, Dep: punct
Token: BTIG, POS: PROPN, Dep: npadvmod
Verb and Subject
True


In [132]:
def extract_target_and_rating(provider, text):
    if provider not in patterns:
        raise ValueError(f"No patterns defined for provider: {provider} + text {text}")

    #Get all patterns for price and rating
    primary_rating_pattern = patterns[provider].get("primary_rating_pattern")
    secondary_rating_pattern = patterns[provider].get("secondary_rating_pattern")
    primary_price_pattern = patterns[provider].get("primary_price_pattern")
    secondary_price_pattern = patterns[provider].get("secondary_price_pattern")

    # Extract rating
    rating = None
    rating_match = re.search(primary_rating_pattern, text)
    if rating_match:
        rating = rating_match.group(1)
    if rating == None:
        if secondary_rating_pattern:
            rating_match = re.search(secondary_rating_pattern,text)
            if rating_match:
                rating = rating_match.group(1)

    # Try the primary pattern
    price = None
    match = re.search(primary_price_pattern, text)
    if match:
        price = match.group(1)  # Return the first capture group

    # If no match and a secondary pattern exists, try the secondary pattern
    if price == None:
        if secondary_price_pattern:
            match = re.search(secondary_price_pattern, text)
            if match:
                price = match.group(1)  # Return the first capture group

    return rating, price

In [138]:
def check_validity(paragraph):
    # Parse the paragraph
    paragraph = re.sub(r"\s+", " ", paragraph).strip()
    doc = nlp(paragraph)
        
    for sent in doc.sents:
        has_verb = False
        has_subject = False
        
        for token in sent:
            # Check for a verb
            if token.pos_ in {"VERB", "AUX"}:
                has_verb = True
            # Check for a subject
            if token.dep_ in {"nsubj", "nsubjpass"}:
                has_subject = True
        
        # If both a verb and a subject are found, the sentence is valid
        if has_verb and has_subject:
            return True
        
        # At least one word with 5+ letters and all upper case
        if re.search(r"[A-Z]{5,}", paragraph):
            return True
    return False

def filter_valid_paragraphs(paragraphs):
    valid_paragraphs = []

    for paragraph in paragraphs:
        if check_validity(paragraph):  # Validate each paragraph
            valid_paragraphs.append(paragraph)
    return valid_paragraphs


def extract_text_with_format(page, provider):
    #Get all patterns for price and rating
    primary_rating_pattern = patterns[provider].get("primary_rating_pattern")
    secondary_rating_pattern = patterns[provider].get("secondary_rating_pattern")
    primary_price_pattern = patterns[provider].get("primary_price_pattern")
    secondary_price_pattern = patterns[provider].get("secondary_price_pattern")
  
    primary_font_size_pattern = patterns[provider].get("primary_font_size_pattern")
    secondary_font_size_pattern = patterns[provider].get("secondary_font_size_pattern")  # Secondary size pattern

    primary_font_type_pattern = patterns[provider].get("primary_font_type_pattern")
    secondary_font_type_pattern = patterns[provider].get("secondary_font_type_pattern")  # Secondary type pattern

    primary_ending_pattern = patterns[provider].get("primary_ending_pattern")

    # Extract words with font and size details
    words = page.extract_words(extra_attrs=["fontname", "size"])

    # Round text sizes to 3 decimal places (because when printing it showed e.g. 7.9200000016)
    for word in words:
        if "size" in word and word["size"] is not None:
            word["size"] = round(word["size"], 2)

    # Sort words by vertical and horizontal position
    words.sort(key=lambda w: (w["top"], w["x0"]))

    paragraphs = [] 
    current_paragraph = []  
    current_top = None 

    rating = None
    price = None

    # Lookahead buffer for multi-word patterns
    lookahead_buffer = []

    for word in words:
        #Filter words by font and size: Match both type and size together
        #print(word["text"])
        #print(word["fontname"])
        #print(word["size"])
        
         # Build lookahead buffer
        lookahead_buffer.append(word["text"])
        if len(lookahead_buffer) > 10:
            lookahead_buffer.pop(0)
        buffer_text = " ".join(lookahead_buffer)

        # Extract rating
        rating_match = re.search(primary_rating_pattern, buffer_text)
        if rating_match:
            rating = rating_match.group(1)
        if rating == None:
            if secondary_rating_pattern:
                rating_match = re.search(secondary_rating_pattern,buffer_text)
                if rating_match:
                    rating = rating_match.group(1)

        # Try the primary pattern
        match = re.search(primary_price_pattern, buffer_text)
        if match:
            price = match.group(1)  # Return the first capture group

        # If no match and a secondary pattern exists, try the secondary pattern
        if price == None:
            if secondary_price_pattern:
                match = re.search(secondary_price_pattern, buffer_text)
                if match:
                    price = match.group(1)  # Return the first capture group


        if len(lookahead_buffer) > 10:
            lookahead_buffer.pop(0)



        if re.search(primary_ending_pattern, buffer_text):
            return paragraphs, True, rating,price  # Stop processing further


        is_primary_match = (re.match(primary_font_type_pattern, word["fontname"]) and word["size"] == primary_font_size_pattern)
        is_secondary_match = (secondary_font_type_pattern and re.match(secondary_font_type_pattern, word["fontname"]) and word["size"] == secondary_font_size_pattern)

        if not (is_primary_match or is_secondary_match):
            continue

        # Check if the word belongs to the same paragraph
        if current_top is None or abs(word["top"] - current_top) < 13:  # Adjust threshold for line grouping
            current_paragraph.append(word["text"])
        else:
            # New paragraph starts
            paragraphs.append(" ".join(current_paragraph))
            current_paragraph = [word["text"]]

        # Update the current top position
        current_top = word["top"]
        
    # Add the last paragraph
    if current_paragraph:
        paragraphs.append(" ".join(current_paragraph))

    #print(f"Paragraphs: {paragraphs}")
    return filter_valid_paragraphs(paragraphs), False, rating,price


pdf_path = "/Users/oskarroeske/Masterthesis/preprocessing/reports/20151030_BTIG_AAPL_Apple-_Inc..pdf"
provider = "BTIG"

def extract_meta_information_and_paragraph(pdf_path)
if provider:
    all_paragraphs = []  
    first_rating = None
    first_price = None  
    stop_extraction = False

    with pdfplumber.open(pdf_path) as pdf:
        for page_number, page in enumerate(pdf.pages, start=1):
            if stop_extraction:
                break  # Exit the loop if stop_extraction is set
            
            # Extract data from the current page
            paragraphs, stop_extraction, rating, price = extract_informaiton(page, provider=provider)
            
            # Append paragraphs from the current page
            all_paragraphs.extend(paragraphs)

            # Capture the first non-None rating and price
            if rating is not None and first_rating is None:
                first_rating = rating
            if price is not None and first_price is None:
                first_price = price

    # Final values
    print("All Paragraphs:", all_paragraphs)
    print("Final Stop Extraction:", stop_extraction)
    print("First Rating:", first_rating)
    print("First Price:", first_price)

else:
    print(f"No text patterns defined for provider: {provider}")



All Paragraphs: ['Apple’s quarter was fine overall and FQ1 2016 guidance implies EPS (212) 527-3523 growth even at the low end of the range. jgalone@btig.com', 'There is no change to our $10 Fiscal 2016 EPS estimate, which reflects 8% EPS growth and is above consensus.', 'We maintained our estimate of 80 million iPhone units sold in the December quarter, but took down our March estimate to 62.5 million 12 month target units from 66 million.', 'Apple is generating $70 billion of free cash flow annually, which implies an 11% FCF yield.', '52 week range Valuation: Our $160 price target is based on 16x our F2016 EPS estimate of $10. Market Cap (m)', 'Apple’s guidance for the December quarter indicates growth in revenue and EPS, even at the low end of the range. That growth might be low single-digits, but it’s growth nonetheless for a stock that is discounting a fear of revenue and EPS declines. CEO Tim Cook also indicated that iPhone units would grow, which should be welcome news to invest

In [136]:
# NEW VERSION (16.11.2024)
def extract_metadata(filename, ticker_map):
    """
    Extract metadata (date, provider, ticker) from the filename using ticker_map.
    """
    # Extract the date (first 8 digits in the filename)
    date_match = re.match(r"(\d{8})", filename)
    if not date_match:
        return None, None, None, None, None
    date_str = date_match.group(1)
    date = datetime.strptime(date_str, "%Y%m%d")

    # Look for the ticker in the filename
    for ticker in ticker_map.keys():
        ticker_pattern = f"_{ticker}_"  # Ensure ticker is surrounded by underscores
        if ticker_pattern in filename:
            # Extract the portion between date and ticker as the provider
            provider_section = filename.split(f"{date_str}_")[1].split(f"_{ticker}_")[0]
            provider = provider_section.replace('_', ' ')  # Replace underscores with spaces
            # Get company name and industry from the ticker_map
            company_name = ticker_map[ticker]['Company Name']
            industry = ticker_map[ticker]['Industry']
            return date, provider, ticker, company_name, industry

    # If no ticker is found, return None for ticker-related fields
    return date, None, None, None, None

# Define function to clean the extracted text
def clean_text(text):
    # Normalize the text to ensure uniform whitespace
    #text = re.sub(r"\s+", " ", text).strip()

    # Remove email addresses and phone numbers
    text = re.sub(r"\S+@\S+", "", text)
    text = re.sub(r"\(\d{1,3}\)[-\s]?\d{3}[-.\s]?\d{4}", "", text)
    text = re.sub(r"This document is being provided.*?WIRT BERLIN\.", "", text, flags=re.DOTALL | re.IGNORECASE)    # Replace multiple newlines and tabs with a single space
    return text


# Define function to process all PDFs in the directory and store data in DataFrame
def extract_text_from_all_pdfs_to_dataframe(directory_path, provider_info, ticker_map):
    """Extract text from all PDFs in a specified directory and store in a DataFrame with unique ID."""
    data = []
    id_counter = 1  # Initialize an ID counter
    for filename in os.listdir(directory_path):
        if filename.endswith(".pdf"):  # Process only PDF files
            file_path = os.path.join(directory_path, filename)

             # Extract and clean text from PDF
            paragraphs = extract_text_from_pdf(file_path)
            
            # Extract metadata from filename
            date, provider, ticker, company_name, industry = extract_metadata(filename,ticker_map=ticker_map)
            
            text = clean_text(text)  # Apply the clean_text function
            cleaned_text = remove_after_ending_keyword(text, provider)
            
            # Add extracted data to the list
            data.append({
                "ID": id_counter,  # Unique ID
                "filename": filename,
                "date": date,
                "provider": provider,
                "ticker": ticker,
                "company_name": company_name,
                "industry": industry,
                "text": paragraphs
            })
            id_counter += 1  # Increment the ID counter for the next row

    # Create a DataFrame from the list of dictionaries
    df = pd.DataFrame(data)
    return df

# Directory path
pdf_directory = "../preprocessing/reports"  # Replace with your actual folder path

# Run the function and store results in a DataFrame
pdf_df = extract_text_from_all_pdfs_to_dataframe(pdf_directory, provider_info, ticker_map)

# Display the resulting DataFrame to confirm
pdf_df.head()


NameError: name 'extract_text_from_pdf' is not defined

In [49]:


#apply to each row
def extract_info(row):
    provider = row['provider']
    text = row['text']
    rating, price = extract_target_and_rating(provider, text)
    return pd.Series({'target_price': price, 'rating': rating})

In [51]:
pdf_df

,ID,filename,date,provider,ticker,company_name,industry,text,target_price,rating
0,1,20201001_Pivotal_Research_Group_AMZN_AMZN-_We_...,2020-10-01,Pivotal Research Group,AMZN,Amazon.com Inc.,Consumer Discretionary,PIVOTAL\nU.S. Equity Research\nInternet & Medi...,"4,500",BUY
1,2,20190730_FinTrust_Investment_Advisors_AMZN_Fin...,2019-07-30,FinTrust Investment Advisors,AMZN,Amazon.com Inc.,Consumer Discretionary,Special Report - Amazon.com Inc. (AMZN) FinTru...,"1,611",HOLD
2,3,20220427_Wells_Fargo_AAPL_AAPL-_F2Q22_Preview_...,2022-04-27,Wells Fargo,AAPL,Apple Inc.,Technology,"\nEquity Research\nCompany Update — April 20, ...",205.00,Overweight
3,4,20150728_BGC_Partners_AMZN_BGC_AMZN_0721015.pdf,2015-07-28,BGC Partners,AMZN,Amazon.com Inc.,Consumer Discretionary,"July 21, 2015\nColin W. Gillis\nTechnology Ana...",475,HOLD
4,5,20180206_BTIG_AAPL_Apple-_Inc..pdf,2018-02-06,BTIG,AAPL,Apple Inc.,Technology,"2 0-May-2010, N/A, N/A, N/A, N/A, N/A, N/A, N/...",198.00,BUY
...,...,...,...,...,...,...,...,...,...,...
89,90,20220121_Wells_Fargo_JPM_JPM-_Downgrade_Rating...,2022-01-21,Wells Fargo,JPM,JPMorgan Chase & Co.,Financials,"\nEquity Research\nRating Change — January 14,...",210.00,Change
90,91,20150804_BGC_Partners_AAPL_BGC_AAPL_07282015.pdf,2015-08-04,BGC Partners,AAPL,Apple Inc.,Technology,"July 28, 2015\nColin W. Gillis\nTechnology Ana...",115,HOLD
91,92,20210217_Barclays_PEP_PEP_4Q20_Earnings_Prep.pdf,2021-02-17,Barclays,PEP,PepsiCo Inc.,Consumer Staples,CORE\nEquity Research | Instant Insights\n10 F...,None,None
92,93,20191218_Barclays_JPM_JPM_CFO_Commentary_at_Se...,2019-12-18,Barclays,JPM,JPMorgan Chase & Co.,Financials,CORE\nEquity Research | Instant Insights\n12 D...,None,None


In [52]:
pdf_df.to_csv("list_of_texts2.csv")

In [128]:
import re
import spacy

# Load the English language model for spaCy
nlp = spacy.load("en_core_web_sm")

def extract_sentences(text):
    """
    Process text to split into sentences.
    """
    # Clean the text first
    cleaned_text = clean_text(text)
    
    # Parse with spaCy
    doc = nlp(cleaned_text)
    
    # Extract sentences as individual strings
    sentences = [sent.text.strip() for sent in doc.sents if len(sent.text.strip()) > 1]
    
    return sentences

# Extract sentences
sentences = extract_sentences(pdf_df["text"][0])

# Display the sentences
for i, sentence in enumerate(sentences[:200]):  # Show the first 10 sentences for inspection
    print(f"Sentence {i+1}: {sentence}")


Sentence 1: PIVOTAL U.S. Equity Research Internet & Media Pivotal Research Group AMZN:
Sentence 2: We Think the Street – Buy- and Sell-side are September 30, 2020 Looking at SOTP Wrong (Us Included).
Sentence 3: We are Reframing Why AMZN is the Best Mega-cap on a Multi-year Basis.
Sentence 4: BOTTOM LINE:
Sentence 5: We and almost every other investor we have spoken to over the years, has been MICHAEL LEVINE framing the AMZN SOTP valuation wrong.
Sentence 6: Amazon advertising is only ~5% of revenues, but is far greater 212-514-4682 contributor to overall non-AWS EBIT margins than the street recognizes.
Sentence 7: Said differently, If advertising was viewed as a stand-along business unit (we will explain why it shouldn’t be), it would represent well north of 300% of 2020E non-AWS EBIT.
Sentence 8: Amazon (AMZN) Based on our view that there is massive upside by 2024E, we increase our PT to a Street high of $4,500 based on an average of our 2024 SOTP and 30x our 2024 EBIT “power” of $91

# Remove Outlier Words

In [129]:
"""from transformers import pipeline

# Initialize a masked language model pipeline (using BERT)
fill_mask = pipeline("fill-mask", model="bert-base-uncased")

def remove_outlier_words(sentence):
    words = sentence.split()  # Split the sentence into words
    cleaned_sentence = []

    for i, word in enumerate(words):
        # Replace the current word with a [MASK] token
        masked_sentence = " ".join(words[:i] + ["[MASK]"] + words[i+1:])
        
        # Get predictions for the masked word
        predictions = fill_mask(masked_sentence)
        
        # Check if the original word is among the top predicted words
        top_predictions = [pred['token_str'] for pred in predictions]
        if word.lower() in top_predictions:
            cleaned_sentence.append(word)  # Keep the word if it fits in the context

    return " ".join(cleaned_sentence)  # Rebuild the cleaned sentence

# Example sentence
sentence = " Shipping losses continue to be an issue for 365 the company facing razor thin profitability, but it is a mild positive that losses have stabilized as Price Target ($) percent of total revenue in the last two quarters."
cleaned_sentence = remove_outlier_words(sentence)

print("Cleaned Sentence:", cleaned_sentence)"""

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exc

Cleaned Sentence: losses continue to be an issue for the company thin but it is a that have of total revenue in the last two


In [7]:
print(pdf_df)
#pdf_df.to_csv("list_of_texts.csv")

                                             filename  \
0   20201001_Pivotal_Research_Group_AMZN_AMZN-_We_...   
1   20190730_FinTrust_Investment_Advisors_AMZN_Fin...   
2   20220427_Wells_Fargo_AAPL_AAPL-_F2Q22_Preview_...   
3     20150728_BGC_Partners_AMZN_BGC_AMZN_0721015.pdf   
4                  20180206_BTIG_AAPL_Apple-_Inc..pdf   
5   20210820_Phillip_Securities_AAPL_Apple_Inc_Sup...   
6   20220803_Phillip_Securities_AAPL_Apple_Inc_Man...   
7   20150716_JP_Morgan_AAPL_Rod_Hall-s_Daily_Downl...   
8   20171205_JP_Morgan_AMZN_Holiday_eComm_Update-_...   
9   20190731_Cascend_Securities_-Historical-_AAPL_...   
10  20221230_JP_Morgan_AAPL_Apple-_Public_CAICT_Ch...   
11  20161017_JP_Morgan_AAPL_Rod_Hall-s_Daily_Downl...   
12  20160510_Phillip_Securities_AAPL_Apple_Inc._Th...   
13  20170804_Needham_AMZN_Mixed_Q1-_Investors_Shou...   
14   20141028_BGC_Partners_AAPL_BGC_AAPL_10212014.pdf   
15                 20190107_BTIG_AAPL_Apple-_Inc..pdf   
16   20160803_BGC_Partners_AAPL